In [2]:
import tensorflow as tf
tf.enable_eager_execution()

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
import pandas as pd

In [5]:
df = pd.read_json('bbc-sarcasm.json')

In [6]:
df = df.drop('article_link',axis=True)

In [7]:
df.head()

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [8]:
stopwatch = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ,"s","v"]

In [9]:
X = df['headline']

In [10]:
y = df['is_sarcastic']

# Text Processing 

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [13]:
train_sentences = []
train_label = []
test_sentences = []
test_labels = []

In [14]:
for i , l in zip(X_train , y_train):
    train_sentences.append(i)
    train_label.append(l)
for j , k in zip(X_test , y_test):
    test_sentences.append(j)
    test_labels.append(k)

In [15]:
train_sentences_final = []
test_sentences_final = []

In [16]:
for s in train_sentences:    
    for w in stopwatch:
        a = " "+w+" "
        s = s.replace(a," ")
        s = s.replace("  ", " ")
    train_sentences_final.append(s)    

In [17]:
for s in test_sentences:    
    for w in stopwatch:
        a = " "+w+" "
        s = s.replace(a," ")
        s = s.replace("  ", " ")
    test_sentences_final.append(s)  

# Tokenization

In [18]:
token = Tokenizer(num_words=10000,oov_token="<OOV>")

In [19]:
token.fit_on_texts(train_sentences_final)

In [20]:
word_index = token.index_word

In [21]:
train_seq = token.texts_to_sequences(train_sentences_final)

In [22]:
train_padded = pad_sequences(train_seq, padding='post' , maxlen=12)

In [23]:
test_seq = token.texts_to_sequences(test_sentences_final)

In [24]:
test_padded = pad_sequences(test_seq , padding='post' , maxlen=12)

In [25]:
#train_padded
#test_padded
#Padded sequences is now ready

# Training model 

In [26]:
from tensorflow.keras.layers import Dense , GlobalAvgPool1D , Embedding , Flatten , Dropout
from tensorflow.keras.models import Sequential

In [88]:
model = Sequential()

In [89]:
model.add(Embedding(10000,8,input_length = 12))

model.add(GlobalAvgPool1D())

model.add(Dense(units = 8,activation = 'relu'))
model.add(Dropout(0.55))

model.add(Dense(units = 4,activation = 'relu'))
model.add(Dropout(0.25))

model.add(Dense(units = 1,activation = 'sigmoid'))

In [90]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [91]:
model.fit(x=train_padded , y=train_label ,
          epochs=10 , validation_data=(test_padded , test_labels))

Train on 18696 samples, validate on 8013 samples
Epoch 1/10
18696/18696 [==============================] - 4s 188us/sample - loss: 0.6596 - acc: 0.6372 - val_loss: 0.5605 - val_acc: 0.7895
Epoch 2/10
18696/18696 [==============================] - 3s 153us/sample - loss: 0.5000 - acc: 0.8192 - val_loss: 0.4721 - val_acc: 0.8118
Epoch 3/10
18696/18696 [==============================] - 3s 149us/sample - loss: 0.3944 - acc: 0.8673 - val_loss: 0.4658 - val_acc: 0.8093
Epoch 4/10
18696/18696 [==============================] - 3s 155us/sample - loss: 0.3242 - acc: 0.8932 - val_loss: 0.5137 - val_acc: 0.8049
Epoch 5/10
18696/18696 [==============================] - 3s 150us/sample - loss: 0.2781 - acc: 0.9105 - val_loss: 0.5869 - val_acc: 0.8069
Epoch 6/10
18696/18696 [==============================] - 3s 154us/sample - loss: 0.2502 - acc: 0.9189 - val_loss: 0.6405 - val_acc: 0.8039
Epoch 7/10
18696/18696 [==============================] - 3s 150us/sample - loss: 0.2269 - acc: 0.9285 - val_lo

In [92]:
from sklearn.metrics import classification_report , confusion_matrix

In [93]:
preds = model.predict_classes(test_padded)

In [94]:
print(classification_report(test_labels , preds))

              precision    recall  f1-score   support

           0       0.81      0.84      0.82      4485
           1       0.78      0.75      0.77      3528

    accuracy                           0.80      8013
   macro avg       0.80      0.79      0.80      8013
weighted avg       0.80      0.80      0.80      8013



In [95]:
print(confusion_matrix(test_labels , preds))

[[3759  726]
 [ 878 2650]]


In [96]:
to_test = ["thousands turn out for empire state building’s annual no-hassle suicide day"]#New sarcastic sentence

In [97]:
seq_test = token.texts_to_sequences(to_test)

In [98]:
seq_test_padded = pad_sequences(seq_test , padding='post' , maxlen=12 )

In [99]:
new_pred = model.predict_classes(seq_test_padded)

In [100]:
new_pred #Prediction 1->is_sarcastic

array([[1]])

In [158]:
to_test1 = ["trump signs sanctions law over China treatment of Uighurs"]#Real News

In [159]:
seq_test1 = token.texts_to_sequences(to_test1)

In [160]:
seq_test_padded1 = pad_sequences(seq_test1 , padding='post' , maxlen=12 )

In [161]:
new_pred1 = model.predict_classes(seq_test_padded1)

In [162]:
new_pred1 #Prediction 0->is_real

array([[0]])

In [170]:
import io

In [165]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 8)


In [171]:
out_v = io.open('vecs1.tsv','w',encoding='utf-8')
out_m = io.open('meta1.tsv','w',encoding='utf-8')
for word_num in range(1,10000):
    word = word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) +"\n")
out_v.close()
out_m.close()